In [119]:
from Librerias_SAP import SAP_GUI
import win32com.client
import pandas as pd
import subprocess
from time import sleep
from datetime import datetime
from datetime import timedelta

In [120]:
def Clean_Columns(List):
    Set_Columns=[i.strip() for i in List.columns]   #Alert in list
    clean_column=[i if "Unnamed" in i else None for i in List.columns]
    clean_column = list(filter(lambda x: x is not None, clean_column)) 
    List=List.set_axis(Set_Columns, axis=1).drop(clean_column,axis=1)  #Drop Unammed: 0
    return(List)

"""--------------------------------FUNCION PARA LIMPIAR NUMEROS A FLOAT--------------------------------------------------------"""

def Clean_column_number(column):
    column=column.apply(lambda x: float(str(x).strip().replace(",","")))
    return(column)

def default_column(default_columns,dataframe):     #Parametros (default_columns: Columnas predeterminadas,dataframe:)
    diccionay_default_column={}
    if len(default_columns)==len(dataframe.columns):
        for i in range(0,len(default_columns)):
            diccionay_default_column[dataframe.columns[i]]=default_columns[i]   
        dataframe=dataframe.rename(columns=diccionay_default_column)   #Remplazo las columnas con las de default
    else:
        None   #asumo columnas originales como estandar
    return(dataframe)

def Date_Null(dataframe,date):
    filter=(date!="00.00.0000")
    dataframe=dataframe.loc[filter]
    return(dataframe)

In [121]:
""" Search MB52 bulk"""
"""----------------------------Inciar session----------------------------------------------------"""
# Insert User name and password
Keys=pd.read_excel("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Claves\\Keys.xlsx")
user=Keys["User"][1]
password=Keys["Password"][1]
# Initialize session
Iniciar=SAP_GUI.SessionSAP(user,password)
# Start transsación
Transsacion="mb52"
provision="GRANELES"   
variant="AUXNVOSPT"
Buscar_MB52_GRANELES=SAP_GUI.Search_MB52(Transsacion,Iniciar,provision,variant)


In [122]:
Name_GRANELES="GRANELES"  # Name file
SAP_GUI.Export_TXT2(Name_GRANELES,Iniciar)

In [123]:
Ruta_GRANELES="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"+str(Name_GRANELES)+".txt"
data_GRANELES = pd.read_csv(Ruta_GRANELES, skiprows=1, delimiter='\t')

In [124]:
data_GRANELES=Clean_Columns(data_GRANELES)

In [126]:
default_column_MB52=['Material', 'Texto breve de material', 'Gpo.artíc.', 'Ce.', 'Alm.',
       'Lote', 'UMB', 'Libre utilización', 'En control calidad',
       'Valor libre util.', 'Valor en insp.cal.', 'Bloqueado',
       'Stock no libre', 'Valor stock bloq.', 'Valor no libre', 'Cad./FPC'] #Disposicion PROXI A VENCER


In [127]:
data_GRANELES=default_column(default_column_MB52,data_GRANELES) #Grabeles limpiar
data_GRANELES=Date_Null(data_GRANELES,data_GRANELES['Cad./FPC'])


In [128]:

"""-------------------------Filtro para eliminar subtotales y combertir fechas-------------------------------"""
data_GRANELES=data_GRANELES[data_GRANELES.Material.notnull()]
data_GRANELES["Cad./FPC"]=data_GRANELES["Cad./FPC"].apply(lambda x: datetime.strptime(x,'%d.%m.%Y'))  #Change columns str to datetime



In [129]:
Data_Denominacion=Clean_Columns(pd.read_csv("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\Denominacion_Articulos.txt",delimiter='\t'))[["Grupo art.","Denominación 2 del gr.artículos"]]
Data_Denominacion=Data_Denominacion.rename(columns={"Denominación 2 del gr.artículos":"Denom.gr.artíc."})   #casa
data_GRANELES=pd.merge(Data_Denominacion,data_GRANELES,how="right",left_on="Grupo art.",right_on="Gpo.artíc.")

In [130]:
Data_Denominacion

,Grupo art.,Denom.gr.artíc.
0,S049,Servicios
1,S048,Servicios
2,S047,Servicios
3,C050,Compras
4,B036,Compras
...,...,...
483,R018,Mantenimiento
484,C141,VERP DECORADO RIONEG
485,C100,Compras
486,C027,Wellonda


In [131]:
"""---------------------------------Filtro graneles vencidos-------------------"""
now=datetime.now()
month_vencido=now-timedelta(days=30) 
mask_vencido=(data_GRANELES["Cad./FPC"]<=now) & (data_GRANELES["Cad./FPC"]>=month_vencido)
data_GRANELES_vencidos=data_GRANELES.loc[mask_vencido]
#Sort values
data_GRANELES_vencidos=data_GRANELES_vencidos.sort_values(by=["Cad./FPC"],ascending=False)
data_GRANELES_vencidos=data_GRANELES_vencidos.reset_index(drop=True)
data_GRANELES_vencidos["Cad./FPC"]=data_GRANELES_vencidos["Cad./FPC"].dt.date   #convet datetime to date 

In [99]:
data_GRANELES_vencidos

,Grupo art.,Denom.gr.artíc.,Material,Texto breve de material,Gpo.artíc.,Ce.,Alm.,Lote,UMB,Libre utilización,En control calidad,Valor libre util.,Valor en insp.cal.,Valor stock bloq.,Valor no libre,Cad./FPC
0,C014,GR. Prebel,PB0078131,GEL CAPILAR MÁXIMA FIJACIÓN (RW),C014,1000.0,1001.0,10899685,KG,0,0,0,0,0,"1,526,010",2023-06-22
1,C014,GR. Prebel,PB0020133,WIP VIOLET N°2 K7014-J,C014,1000.0,1001.0,10897141,KG,0,0,0,0,0,"354,874",2023-06-21
2,C030,GR. Unilever,PB0093603,EGO GEL EXTREME MAX COL BET,C030,1000.0,1001.0,10901285,KG,0,0,0,0,0,"23,857",2023-06-16
3,C295,GR.KOBA,PB0089717,CREMA EXF. CORP. AVENA - KOBA,C295,1000.0,1001.0,10883619,KG,0,0,0,0,0,"118,061",2023-06-14
4,C295,GR.KOBA,PB0089716,CREMA EXF. CORP. POMGRANE - KOBA,C295,1000.0,1001.0,10895476,KG,0,0,0,0,0,"64,131",2023-06-14
5,C136,GR.Biotecnick,PB0082173,SLN COLOR AMARILLO CIBA,C136,1000.0,1001.0,10886461,KG,0,0,0,0,0,"184,512",2023-06-14
6,C038,GR. Avon,1-67645,AVON TRUE MICELLAR WATER RESTA,C038,1000.0,1001.0,10844582,KG,0,0,0,0,0,"102,732",2023-06-14
7,C014,GR. Prebel,PB0083923,MF PAN STICK BASE 61 ORO VIEJO,C014,1000.0,1001.0,10900899,KG,0,0,0,0,0,"509,881",2023-06-11
8,C014,GR. Prebel,PB0087764,AGUA MICELAR (NF),C014,1000.0,1001.0,10899981,KG,0,0,0,0,0,"796,743",2023-06-09
9,C014,GR. Prebel,PB0077030,DEO PIES AEROSOL AFM ANTIBACTERIAL NF,C014,1000.0,1001.0,10900440,KG,0,0,0,0,0,"4,118,790",2023-06-05


In [132]:
# filter dates between now a 30 month later
now=datetime.now()
month=timedelta(days=6*30)   # Change limiter day(30 days= 1 month)
month=now+month
mask=(data_GRANELES["Cad./FPC"]>=now) & (data_GRANELES["Cad./FPC"]<=month)
data_GRANELES=data_GRANELES.loc[mask]
#Sort values
data_GRANELES=data_GRANELES.sort_values(by=["Cad./FPC","Libre utilización"])
data_GRANELES=data_GRANELES.reset_index(drop=True)
data_GRANELES["Cad./FPC"]=data_GRANELES["Cad./FPC"].dt.date   #convet datetime to date 

In [133]:
data_GRANELES.columns

Index(['Grupo art.', 'Denom.gr.artíc.', 'Material', 'Texto breve de material',
       'Gpo.artíc.', 'Ce.', 'Alm.', 'Lote', 'UMB', 'Libre utilización',
       'En control calidad', 'Valor libre util.', 'Valor en insp.cal.',
       'Bloqueado', 'Stock no libre', 'Valor stock bloq.', 'Valor no libre',
       'Cad./FPC'],
      dtype='object')

In [102]:
"""------------DATAFRAME GRANALES PARA INFORME-----------------------------------------------------------------------"""

data_Graneles_informe=data_GRANELES[['Material', 'Texto breve de material', 'Gpo.artíc.','Denom.gr.artíc.', 'Ce.', 'Alm.',
       'Lote',  'Libre utilización', 'En control calidad','UMB', 'Valor libre util.', 'Valor en insp.cal.','Cad./FPC']]


data_Graneles_Vencidos_informe=data_GRANELES_vencidos[['Grupo art.', 'Denom.gr.artíc.', 'Material', 'Texto breve de material',
       'Gpo.artíc.', 'Ce.', 'Alm.', 'Lote','Bloqueado',
       'Stock no libre', 'UMB','Valor stock bloq.', 'Valor no libre',
       'Cad./FPC']]


data_Graneles_Vencidos_informe
#data_Graneles_informe

,Grupo art.,Denom.gr.artíc.,Material,Texto breve de material,Gpo.artíc.,Ce.,Alm.,Lote,UMB,Valor stock bloq.,Valor no libre,Cad./FPC
0,C014,GR. Prebel,PB0078131,GEL CAPILAR MÁXIMA FIJACIÓN (RW),C014,1000.0,1001.0,10899685,KG,0,"1,526,010",2023-06-22
1,C014,GR. Prebel,PB0020133,WIP VIOLET N°2 K7014-J,C014,1000.0,1001.0,10897141,KG,0,"354,874",2023-06-21
2,C030,GR. Unilever,PB0093603,EGO GEL EXTREME MAX COL BET,C030,1000.0,1001.0,10901285,KG,0,"23,857",2023-06-16
3,C295,GR.KOBA,PB0089717,CREMA EXF. CORP. AVENA - KOBA,C295,1000.0,1001.0,10883619,KG,0,"118,061",2023-06-14
4,C295,GR.KOBA,PB0089716,CREMA EXF. CORP. POMGRANE - KOBA,C295,1000.0,1001.0,10895476,KG,0,"64,131",2023-06-14
5,C136,GR.Biotecnick,PB0082173,SLN COLOR AMARILLO CIBA,C136,1000.0,1001.0,10886461,KG,0,"184,512",2023-06-14
6,C038,GR. Avon,1-67645,AVON TRUE MICELLAR WATER RESTA,C038,1000.0,1001.0,10844582,KG,0,"102,732",2023-06-14
7,C014,GR. Prebel,PB0083923,MF PAN STICK BASE 61 ORO VIEJO,C014,1000.0,1001.0,10900899,KG,0,"509,881",2023-06-11
8,C014,GR. Prebel,PB0087764,AGUA MICELAR (NF),C014,1000.0,1001.0,10899981,KG,0,"796,743",2023-06-09
9,C014,GR. Prebel,PB0077030,DEO PIES AEROSOL AFM ANTIBACTERIAL NF,C014,1000.0,1001.0,10900440,KG,0,"4,118,790",2023-06-05


In [134]:
data_Graneles_Vencidos_informe.columns

Index(['Grupo art.', 'Denom.gr.artíc.', 'Material', 'Texto breve de material',
       'Gpo.artíc.', 'Ce.', 'Alm.', 'Lote', 'UMB', 'Valor stock bloq.',
       'Valor no libre', 'Cad./FPC'],
      dtype='object')

In [135]:
"""-----------------------------GENERO SUBTOTALES DE GRANELES------------------------------------------------------------"""

data_GRANELES_Libre=Clean_column_number(data_GRANELES["Valor libre util."]).sum()
data_GRANELES_Calidad=Clean_column_number(data_GRANELES['Valor en insp.cal.']).sum()
Total_Graneles=data_GRANELES_Libre+data_GRANELES_Calidad
Total_Graneles="${:,.2f}".format(Total_Graneles)

In [136]:
"""-----------------------------GENERO SUBTOTALES DE GRANELES VENCIDOS--------------------------------------------------------"""
data_GRANELES_Stock_Bloqueado=Clean_column_number(data_GRANELES_vencidos['Valor stock bloq.']).sum()
data_GRANELES_No_Libre=Clean_column_number(data_GRANELES_vencidos['Valor no libre']).sum()
Total_Graneles_Vencidos=data_GRANELES_Stock_Bloqueado+data_GRANELES_No_Libre
Total_Graneles_Vencidos="${:,.2f}".format(Total_Graneles_Vencidos)

In [137]:
Total_Graneles_Vencidos

'$11,012,956.00'

In [143]:
""" Search MB52 raw material"""

# Start transsación
Transsacion="mb52"
provision="MATERIA PRIMA"  
variant="AUXNVOSPT"
Buscar_MB52_MATERIAS=SAP_GUI.Search_MB52(Transsacion,Iniciar,provision,variant)
# Save file



Name_Materias="MATERIAS PRIMAS"  # Revisar suele no guardar
SAP_GUI.Export_TXT2(Name_Materias,Iniciar)
# Pass the route and read file 
Ruta_Materias="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"+str(Name_Materias)+".txt"
data_Materias=pd.read_csv(Ruta_Materias,delimiter="\t")
# Change, drop and standardize columns
data_Materias=Clean_Columns(data_Materias)
data_Materias=default_column(default_column_MB52,data_Materias) 
data_Materias=Date_Null(data_Materias,data_Materias['Cad./FPC'])


"""-------------------------Filtro para eliminar subtotales y combertir fechas-------------------------------"""
data_Materias=data_Materias[data_Materias.Material.notnull()]
data_Materias["Cad./FPC"]=data_Materias["Cad./FPC"].apply(lambda x: datetime.strptime(str(x),'%d.%m.%Y'))  #Change columns str to datetime

data_Materias=pd.merge(Data_Denominacion,data_Materias,how="right",left_on="Grupo art.",right_on="Gpo.artíc.")


In [144]:
"""---------------------------Buscamos las clases de material------------- """
Data_Material_Total=data_Materias
Lista_tipo_Materias=data_Materias["Gpo.artíc."].drop_duplicates().reset_index(drop=True)


In [145]:
"""--------------------------------------MATERIAS PRIMAS VENCIDAS-------------------------"""

mask_vencido=(data_Materias["Cad./FPC"]<=now) & (data_Materias["Cad./FPC"]>=month_vencido)
data_Materias_vencidos=data_Materias.loc[mask_vencido]
#Sort values
data_Materias_vencidos=data_Materias_vencidos.sort_values(by=["Cad./FPC"],ascending=False)
data_Materias_vencidos=data_Materias_vencidos.reset_index(drop=True)
data_Materias_vencidos["Cad./FPC"]=data_Materias_vencidos["Cad./FPC"].dt.date   #convet datetime to date 


In [110]:
data_Materias_vencidos

,Grupo art.,Denom.gr.artíc.,Material,Texto breve de material,Gpo.artíc.,Ce.,Alm.,Lote,UMB,Libre utilización,En control calidad,Valor libre util.,Valor en insp.cal.,Valor stock bloq.,Valor no libre,Cad./FPC
0,C041,MP. Natura,PB0077384,357686 AVOBENZONE - DSM,C041,1000.0,1001.0,10893581.0,KG,0,0,0,0,0,"924,333",2023-06-22
1,C026,MP. Unilever,U121460,CLORO METIL ISO TIAZOLINONA,C026,1000.0,1001.0,10891409.0,KG,0,0,0,0,0,0,2023-06-22
2,GRIN1,MP Avon,R159345,SPORT CLUB BARCELONA UAG09565-,GRIN1,1000.0,1001.0,10867523.0,G,0,0,0,0,0,0,2023-06-22
3,C274,MP.BEIERSDORF,0018516900,EUTANOL G,C274,1000.0,1001.0,10885537.0,G,0,0,0,0,0,"12,031,417",2023-06-21
4,C041,MP. Natura,PB0077519,50276607 ESSEN. AROM. LN02 NATURA,C041,1000.0,1001.0,10868981.0,KG,0,0,0,0,0,"149,701",2023-06-20
5,C094,MP.Omnilife,PB0050484,BEAUPLEX VH,C094,1000.0,1001.0,10816967.0,KG,0,0,0,0,0,"7,760,064",2023-06-20
6,C094,MP.Omnilife,PB0050317,PHYTEXCELL GINSENG,C094,1000.0,1001.0,10800248.0,KG,0,0,0,0,0,"1,276,742",2023-06-19
7,GRIN1,MP Avon,R0937,POE (26M)-POP (29M) POLYME,GRIN1,1000.0,1001.0,10876571.0,G,0,0,0,0,0,0,2023-06-19
8,C274,MP.BEIERSDORF,PB0084144,SODIUM ASCORBYL PHOSPHATE12084,C274,1000.0,1001.0,10875220.0,KG,0,0,0,0,0,"1,882,284",2023-06-19
9,C221,MP.Jerónimo Martens,100RX01920,50038722 DMDM HYDANTOIN,C221,1000.0,1001.0,10874196.0,KG,0,0,0,0,0,"1,576,656",2023-06-19


In [146]:

mask=(data_Materias["Cad./FPC"]>=now) & (data_Materias["Cad./FPC"]<=month)
data_Materias=data_Materias.loc[mask]
data_Materias=data_Materias.sort_values(by=["Cad./FPC","Libre utilización"]).reset_index(drop=True)
data_Materias["Cad./FPC"]=data_Materias["Cad./FPC"].dt.date

#Close Session
#SAP_GUI.Close_session(Iniciar)


"""------------DATAFRAME MATERIAS PARA INFORME-----------------------------------------------------------------------"""
data_Materia_informe=data_Materias[['Material', 'Texto breve de material', 'Gpo.artíc.','Denom.gr.artíc.', 'Ce.', 'Alm.',
       'Lote',  'Libre utilización', 'En control calidad','UMB', 'Valor libre util.', 'Valor en insp.cal.','Cad./FPC']]

data_Materia_informe_vencidos=data_Materias_vencidos[['Grupo art.', 'Denom.gr.artíc.', 'Material', 'Texto breve de material',
       'Gpo.artíc.', 'Ce.', 'Alm.', 'Lote','Bloqueado',
       'Stock no libre', 'UMB','Valor stock bloq.', 'Valor no libre',
       'Cad./FPC']]


In [112]:
data_Materia_informe_vencidos

,Grupo art.,Denom.gr.artíc.,Material,Texto breve de material,Gpo.artíc.,Ce.,Alm.,Lote,UMB,Valor stock bloq.,Valor no libre,Cad./FPC
0,C041,MP. Natura,PB0077384,357686 AVOBENZONE - DSM,C041,1000.0,1001.0,10893581.0,KG,0,"924,333",2023-06-22
1,C026,MP. Unilever,U121460,CLORO METIL ISO TIAZOLINONA,C026,1000.0,1001.0,10891409.0,KG,0,0,2023-06-22
2,GRIN1,MP Avon,R159345,SPORT CLUB BARCELONA UAG09565-,GRIN1,1000.0,1001.0,10867523.0,G,0,0,2023-06-22
3,C274,MP.BEIERSDORF,0018516900,EUTANOL G,C274,1000.0,1001.0,10885537.0,G,0,"12,031,417",2023-06-21
4,C041,MP. Natura,PB0077519,50276607 ESSEN. AROM. LN02 NATURA,C041,1000.0,1001.0,10868981.0,KG,0,"149,701",2023-06-20
5,C094,MP.Omnilife,PB0050484,BEAUPLEX VH,C094,1000.0,1001.0,10816967.0,KG,0,"7,760,064",2023-06-20
6,C094,MP.Omnilife,PB0050317,PHYTEXCELL GINSENG,C094,1000.0,1001.0,10800248.0,KG,0,"1,276,742",2023-06-19
7,GRIN1,MP Avon,R0937,POE (26M)-POP (29M) POLYME,GRIN1,1000.0,1001.0,10876571.0,G,0,0,2023-06-19
8,C274,MP.BEIERSDORF,PB0084144,SODIUM ASCORBYL PHOSPHATE12084,C274,1000.0,1001.0,10875220.0,KG,0,"1,882,284",2023-06-19
9,C221,MP.Jerónimo Martens,100RX01920,50038722 DMDM HYDANTOIN,C221,1000.0,1001.0,10874196.0,KG,0,"1,576,656",2023-06-19


In [113]:
"""-----------------------------GENERO SUBTOTALES MATERIAS PRIMAS------------------------------------------------------------"""

data_Materia_Libre=Clean_column_number(data_Materias["Valor libre util."]).sum()
data_Materia_Calidad=Clean_column_number(data_Materias['Valor en insp.cal.']).sum()
Total_Materia=data_Materia_Libre+data_Materia_Calidad
Total_Materia="${:,.2f}".format(Total_Materia)

In [114]:
"""-----------------------------GENERO SUBTOTALES DE MATERIAS PRIMAS--------------------------------------------------------"""
data_Materia_Stock_Bloqueado=Clean_column_number(data_Materias_vencidos['Valor stock bloq.']).sum()
data_Materia_No_Libre=Clean_column_number(data_Materias_vencidos['Valor no libre']).sum()
Total_Materia_Vencidos=data_Materia_Stock_Bloqueado+data_Materia_No_Libre
Total_Materia_Vencidos="${:,.2f}".format(Total_Materia_Vencidos)

In [115]:
Total_Materia_Vencidos

'$193,054,014.00'

In [116]:
data_Materias

,Grupo art.,Denom.gr.artíc.,Material,Texto breve de material,Gpo.artíc.,Ce.,Alm.,Lote,UMB,Libre utilización,En control calidad,Valor libre util.,Valor en insp.cal.,Valor stock bloq.,Valor no libre,Cad./FPC
0,C242,MP.HENKEL,H2561691,EXTRA FINE GRANULATED SUGAR,C242,1000.0,1001.0,10873278.0,G,74,0,0,0,0,0,2023-06-23
1,C242,MP.HENKEL,H2561691,EXTRA FINE GRANULATED SUGAR,C242,1000.0,1001.0,10883672.0,G,87,0,0,0,0,0,2023-06-23
2,C242,MP.HENKEL,H2561691,EXTRA FINE GRANULATED SUGAR,C242,1000.0,1001.0,10888146.0,G,136,0,0,0,0,0,2023-06-23
3,C242,MP.HENKEL,H1806411,"CROSILK 10,000 - PF",C242,1000.0,1001.0,10888145.0,KG,0.024,0,0,0,0,0,2023-06-23
4,C242,MP.HENKEL,H1806411,"CROSILK 10,000 - PF",C242,1000.0,1001.0,10884021.0,KG,0.055,0,0,0,0,0,2023-06-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,C041,MP. Natura,PB0090145,50373100 FRAG COCKTAILIFRAAR 264586,C041,1000.0,1001.0,10895153.0,KG,164.274,0,"24,212,345",0,0,0,2023-12-17
658,C329,MP.MILAGROS,PB0092451,ÁCIDO GLUTAMICO,C329,1000.0,1001.0,10884443.0,KG,85.300,0,"2,431,050",0,0,0,2023-12-18
659,C329,MP.MILAGROS,PB0092451,ÁCIDO GLUTAMICO,C329,1000.0,1001.0,10886308.0,KG,224.915,0,"6,410,076",0,0,0,2023-12-18
660,GRIN1,MP Avon,3739,PPG 20 METHYL GLUCOSE ETH,GRIN1,1000.0,1001.0,10859744.0,G,625,0,0,0,0,0,2023-12-19


In [117]:
"""Send email"""

correos="prac.ingindustrial2@prebel.com.co"

def send_emails(*args,emails="",htmlbody="",subject=""):
    email=emails
    outlook=win32com.client.Dispatch("outlook.application")
    mail=outlook.CreateItem(0)
    mail.Subject=subject+" "+datetime.now().strftime('%#d %b %Y %H:%M')
    mail.To=email
    mail.HTMLBody=htmlbody.format(*args)
    mail.Send()

def style_df(df):
    return df.style \
        .set_table_styles([{'selector': "table,tr,th,td", 'props': [("border", "1px solid"), ('color', '#000'),("text-align","center")]}]) \
        .highlight_between(subset=["Cad./FPC"],color='#FF5733',left=now.date(),right=(now.date()+timedelta(days=7))) \

html="""
    <h2 style="text-align: center">REPORTE CADUCIDAD DE GRANELES</h2>
    <p> Por medio del presente informe se evidencia los graneles segun el tipo de material que estan proximos a vencer durante un mes a partir de la 
        fecha actual, ademas de las cantidades con su respectiva unidad de medida que estan en libre utilización.</p>

    <h4 style="color: black;" > Advertencia: El valor económico potencial en riesgo de vencimiento es de: </h4>
    
    <h1 style="color: red;" > {1} </h1>

    <div"> {0} </div>

    <h4 style="color: red;" > Advertencia: Los graneles a vencer en los proximos 7 días se resaltan en rojo</h4>

    <p> Anticipo sinceros agradecimientos. </p>
 """

html2="""
    <h2 style="text-align: center"> REPORTE CADUCIDAD DE MATERIAS PRIMAS</h2>
    <p> Por medio del presente informe se evidencia las materias primas segun el tipo de material que estan proximos a vencer durante un mes a partir de la 
        fecha actual, ademas de las cantidades con su respectiva unidad de medida que estan en libre utilización.</p>

    <h4 style="color: black;" > Advertencia: El valor económico potencial en riesgo de vencimiento es de: </h4>
    
    <h1 style="color: red;" > {1} </h1>
    
    <div"> {0} </div>

    <h4 style="color: red;" > Advertencia: Las materias primas proximas a vencer en los proximos 7 días se resaltan en rojo</h4>

    <p> Anticipo sinceros agradecimientos. </p>
 """

Send = style_df(data_Graneles_informe)  #Style between LI and LS
Send2=style_df(data_Materia_informe)     #Style between LI and LS



# Definir el diccionario de formato
try:
    formato_Graneles = {'Ce.': '{:.0f}',"Alm.":'{:.0f}'}
    formato_material= {'Ce.': '{:.0f}',"Alm.":'{:.0f}',"Lote":'{:.0f}'}
    # Aplicar el formato a la columna 'Altura'
    Send = Send.format(formato_Graneles)
    Send2=Send2.format(formato_material)
except:
    None

send_emails(Send.to_html(),Total_Graneles,emails=correos,htmlbody=html,subject="REPORTE GRANELES PROXIMOS A VENCER")
send_emails(Send2.to_html(),Total_Materia,emails=correos,htmlbody=html2,subject="REPORTE MATERIAS PRIMAS PROXIMOS A VENCER")

In [69]:
send_emails(Send.to_html(),Total_Graneles,emails=correos,htmlbody=html,subject="REPORTE GRANELES PROXIMOS A VENCER")

In [151]:
data_Materia_informe_vencidos["Alm."]=data_Materia_informe_vencidos["Alm."].astype(int)
data_Materia_informe_vencidos["Lote"]=data_Materia_informe_vencidos["Lote"].astype(int)



C:\Users\prac.ingindustrial2\AppData\Local\Temp\ipykernel_16396\1200802570.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_Materia_informe_vencidos["Alm."]=data_Materia_informe_vencidos["Alm."].astype(int)
C:\Users\prac.ingindustrial2\AppData\Local\Temp\ipykernel_16396\1200802570.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_Materia_informe_vencidos["Lote"]=data_Materia_informe_vencidos["Lote"].astype(int)


0     10893581
1     10891409
2     10867523
3     10885537
4     10868981
5     10816967
6     10800248
7     10876571
8     10875220
9     10874196
10    10899491
11    10874434
12    10873611
13    10834478
14    10826831
15    10872450
16    10873624
17    10874283
18    10872442
19    10895785
20    10868221
21    10866365
22    10862546
23    10895843
24    10864990
25    10892850
26    10732092
27    10875138
28    10838869
29    10870144
30    10872132
31    10812051
32    10811338
33    10872120
34    10872119
35    10849622
36    10837430
37    10843506
38    10844722
39    10784613
40    10854555
41    10766527
42    10806493
43    10836888
44    10839853
45    10857000
46    10864317
47    10867391
48    10720356
49    10849560
50    10842602
51    10845701
52    10865777
53    10866367
54    10840440
55    10842726
56    10842605
57    10891397
58    10848974
Name: Lote, dtype: int32

In [152]:
"""-------------------------------------Send email material y granel vencidos--------------------------------"""

correos="prac.ingindustrial2@prebel.com.co"

def send_emails(*args,emails="",htmlbody="",subject=""):
    email=emails
    outlook=win32com.client.Dispatch("outlook.application")
    mail=outlook.CreateItem(0)
    mail.Subject=subject+" "+datetime.now().strftime('%#d %b %Y %H:%M')
    mail.To=email
    mail.HTMLBody=htmlbody.format(*args)
    mail.Send()

def style_df(df):
    return df.style \
        .set_table_styles([{'selector': "table,tr,th,td", 'props': [("border", "1px solid"), ('color', '#000'),("text-align","center")]}]) \

html="""
    <h2 style="text-align: center">REPORTE CADUCIDAD DE GRANELES</h2>
    <p> Por medio del presente informe se evidencia los graneles y materias primas vencidas, para su correspondiente análisis. </p>

    <h2 style="color: black;" > GRANELES VENCIDOS</h4>

    <h1 style="color: red;" > {2} </h1>
    <div"> {0} </div>

    <h2 style="color: black;" > MATERIA PRIMA VENCIDA </h4>
    
    <h1 style="color: red;" > {3} </h1>
    <div"> {1} </div>


    <h4> Anticipo sinceros agradecimientos. </h4>
 """
formato_Graneles = {'Ce.': '{:.0f}',"Alm.":'{:.0f}'}


Send = style_df(data_Graneles_Vencidos_informe)  #Style between LI and LS
Send2=style_df(data_Materia_informe_vencidos)     #Style between LI and LS
Send=Send.format(formato_Graneles)
Send2=Send2.format(formato_Graneles)
send_emails(Send.to_html(),Send2.to_html(),Total_Graneles_Vencidos,Total_Materia_Vencidos,emails=correos,htmlbody=html,subject="REPORTE GRANELES Y MARIA PRIMA VENCIDO")
